# Testes de treinamento incremental

A ideia deste notebook é verificar se treinar os algoritmos de MAB durante as interações de teste geram alguma melhora para eles, comparando-os com os resultados de um algoritmo estático (ALS).

In [1]:
import pandas as pd
from mab2rec import BanditRecommender, LearningPolicy
from mab2rec.pipeline import train
from sklearn.preprocessing import LabelEncoder
import implicit
from scipy.sparse import csr_matrix
from implicit.nearest_neighbours import bm25_weight
import plotly.express as px
import time

train_data = "../data/ml100k/data_train.csv"
test_data = "../data/ml100k/data_test.csv"

In [2]:
def initial_train(df, num_users, num_items):
    FACTORS = 10
    K1 = 100
    B = 0.8

    # Cria a matriz esparsa
    sparse_matrix = csr_matrix((df['response'], (df['user_id'], df['item_id'])), shape=(num_users, num_items))
    updated_sparse_matrix = bm25_weight(sparse_matrix, K1=K1, B=B)

    print('Treinando o modelo ALS')
    ALS_model = implicit.als.AlternatingLeastSquares(factors=FACTORS, random_state=1)
    ALS_model.fit(updated_sparse_matrix)

    user_features_list = []

    for user_id in df['user_id'].unique():
        user_factors = ALS_model.user_factors[user_id]
        user_features_list.append([user_id] + list(user_factors))

    df_user_features = pd.DataFrame(user_features_list, columns=['user_id'] + [f'u{i}' for i in range(FACTORS)])

    print('Treinando o modelo LinUCB')
    linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(), top_k=10)
    train(linUCB_model, data=df, user_features=df_user_features)
    

    print('Treinando o modelo LinGreedy')
    linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(), top_k=10)
    train(linGreedy_model, data=df, user_features=df_user_features)

    return ALS_model, linUCB_model, linGreedy_model, df_user_features, sparse_matrix

In [3]:

def test_ALS(ALS_model, sparse_matrix, df_test):
    print('Testing ALS')

    start_time = time.time()
    hits = 0
    for i, interaction in df_test.iterrows():
        ids_recs, _ = ALS_model.recommend(userid=interaction['user_id'], user_items=sparse_matrix[interaction['user_id']], N=10)
        if interaction['item_id'] in ids_recs:
            hits += 1
    

    return hits, hits/len(df_test), time.time() - start_time

In [4]:
def test_non_incremental(mab_algo, algo_name, user_features, df_test):
    print(f'Testing {algo_name}')

    start_time = time.time()
    hits = 0

    contexts = df_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

    recomendations = mab_algo.recommend(contexts)

    df_test = df_test.reset_index(drop=True)

    hits = 0
    for i, interaction in df_test.iterrows():
        if interaction['item_id'] in recomendations[i]:
            hits += 1

    return hits, hits/len(df_test), time.time() - start_time

In [5]:
def test_incremental(mab_algo, algo_name, user_features, df_test, df_test_for_evaluation, batch_size):
    print(f'Testing {algo_name} with batch size {batch_size}')

    start_time = time.time()
    hits = 0

    for i in range(0, len(df_test), batch_size):
        # Fazendo recomendações para teste
        df_batch_test = df_test_for_evaluation.loc[i:i+batch_size-1]
        contexts = df_batch_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        if len(contexts) > 0: # Se não tiver nenhuma interação positiva, não faz sentido fazer recomendações
            recomendations = mab_algo.recommend(contexts)
            if isinstance(recomendations, list) and isinstance(recomendations[0], int):
                # Quando o contexto tem tamanho 1, a recomendação é uma lista, e não uma lista de listas
                recomendations = [recomendations]

            df_batch_test = df_batch_test.reset_index(drop=True)

            for j, interaction in df_batch_test.iterrows():
                if interaction['item_id'] in recomendations[j]:
                    hits += 1
        
        # Treinando com o batch
        df_batch_train = df_test.loc[i:i+batch_size-1]
        contexts = df_batch_train.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        mab_algo.partial_fit(df_batch_train['item_id'], df_batch_train['response'], contexts)

    return hits, hits/len(df_test_for_evaluation), time.time() - start_time

In [6]:
def test(train_size, batchs_sizes):
    results = []
    df_train = pd.read_csv(train_data)
    df_test = pd.read_csv(test_data)

    df_full = pd.concat([df_train, df_test])

    df_full['user_id'] = LabelEncoder().fit_transform(df_full['user_id'])
    df_full['item_id'] = LabelEncoder().fit_transform(df_full['item_id'])

    num_users = df_full['user_id'].nunique()
    num_items = df_full['item_id'].nunique()

    split_index = int(len(df_full) * train_size)
    df_train = df_full[:split_index]
    df_test = df_full[split_index:]

    df_test = df_test[(df_test['user_id'].isin(df_train['user_id'])) & (df_test['item_id'].isin(df_train['item_id']))]
    df_test = df_test.reset_index(drop=True)
    df_test_for_evaluation = df_test[df_test['response'] == 1]

    ALS_model, linUCB_model, linGreedy_model, df_user_features, sparse_matrix = initial_train(df_train, num_users, num_items)

    hits, hr, spent_time = test_ALS(ALS_model, sparse_matrix, df_test_for_evaluation)
    results.append({'algorithm': 'ALS', 'hits': hits, 'hr': hr, 'time': spent_time})

    print('\nTesting non-incremental\n')

    hits, hr, spent_time = test_non_incremental(linUCB_model, 'LinUCB', df_user_features, df_test_for_evaluation)
    results.append({'algorithm': 'LinUCB (non_incremental)', 'hits': hits, 'hr': hr, 'time': spent_time})

    hits, hr, spent_time = test_non_incremental(linGreedy_model, 'LinGreedy', df_user_features, df_test_for_evaluation)
    results.append({'algorithm': 'LinGreedy (non_incremental)', 'hits': hits, 'hr': hr, 'time': spent_time})

    print('\nTesting incremental\n')
    
    for batch_size in batchs_sizes:
        print('Treinando o modelo LinUCB')
        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(), top_k=10)
        train(linUCB_model, data=df_train, user_features=df_user_features)

        hits, hr, spent_time = test_incremental(linUCB_model, 'LinUCB', df_user_features, df_test, df_test_for_evaluation, batch_size)
        results.append({'algorithm': 'LinUCB (incremental)', 'hits': hits, 'hr': hr, 'time': spent_time, 'batch_size': batch_size})

        print('Treinando o modelo LinGreedy')
        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(), top_k=10)
        train(linGreedy_model, data=df_train, user_features=df_user_features)

        hits, hr, spent_time = test_incremental(linGreedy_model, 'LinGreedy', df_user_features, df_test, df_test_for_evaluation, batch_size)
        results.append({'algorithm': 'LinGreedy (incremental)', 'hits': hits, 'hr': hr, 'time': spent_time, 'batch_size': batch_size})
    
    df_results = pd.DataFrame(results)
    df_results = df_results.astype({'hits': int, 'hr': float, 'time': float})
    df_results['test_size'] = round(1 - train_size, 2)
    df_results['test_interactions'] = len(df_test_for_evaluation)
    return df_results

In [ ]:
train_sizes = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.65, 0.60, 0.55, 0.50, 0.45, 0.40, 0.35, 0.30, 0.25, 0.20, 0.15, 0.10, 0.05]
batch_sizes = [1, 5, 10, 100, 1000]

for train_size in train_sizes:
    df_results = test(train_size, batch_sizes)
    df_results.to_csv(f'results/results_{str(train_size)[2:]}.csv', index=False)

In [9]:
all_dfs = []

for train_size in train_sizes:
    all_dfs.append(pd.read_csv(f'results/results_{str(train_size)[2:]}.csv'))
    
df_results = pd.concat(all_dfs)
df_results

,algorithm,hits,hr,time,batch_size,test_size,test_interactions
0,ALS,46,0.153846,0.250584,NaN,0.05,299
1,LinUCB (non_incremental),22,0.073579,0.078525,NaN,0.05,299
2,LinGreedy (non_incremental),22,0.073579,0.058038,NaN,0.05,299
3,LinUCB (incremental),17,0.056856,68.856901,1.0,0.05,299
4,LinGreedy (incremental),16,0.053512,75.320913,1.0,0.05,299
...,...,...,...,...,...,...,...
8,LinGreedy (incremental),589,0.071463,157.516431,10.0,0.95,8242
9,LinUCB (incremental),506,0.061393,24.290873,100.0,0.95,8242
10,LinGreedy (incremental),605,0.073405,23.464792,100.0,0.95,8242
11,LinUCB (incremental),485,0.058845,4.258888,1000.0,0.95,8242


In [10]:
df_results.to_csv('results/concat_results.csv', index=False)

In [11]:
df_results_incremental = df_results.dropna()
df_results_incremental

,algorithm,hits,hr,time,batch_size,test_size,test_interactions
3,LinUCB (incremental),17,0.056856,68.856901,1.0,0.05,299
4,LinGreedy (incremental),16,0.053512,75.320913,1.0,0.05,299
5,LinUCB (incremental),17,0.056856,17.926848,5.0,0.05,299
6,LinGreedy (incremental),16,0.053512,17.997318,5.0,0.05,299
7,LinUCB (incremental),19,0.063545,9.397860,10.0,0.05,299
...,...,...,...,...,...,...,...
8,LinGreedy (incremental),589,0.071463,157.516431,10.0,0.95,8242
9,LinUCB (incremental),506,0.061393,24.290873,100.0,0.95,8242
10,LinGreedy (incremental),605,0.073405,23.464792,100.0,0.95,8242
11,LinUCB (incremental),485,0.058845,4.258888,1000.0,0.95,8242


In [12]:
df_results_incremental['algo_batch'] = df_results_incremental['algorithm'] + ' - ' + df_results_incremental['batch_size'].astype(str)
df_results_incremental

C:\Users\gregf\AppData\Local\Temp\ipykernel_22760\172089932.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results_incremental['algo_batch'] = df_results_incremental['algorithm'] + ' - ' + df_results_incremental['batch_size'].astype(str)


,algorithm,hits,hr,time,batch_size,test_size,test_interactions,algo_batch
3,LinUCB (incremental),17,0.056856,68.856901,1.0,0.05,299,LinUCB (incremental) - 1.0
4,LinGreedy (incremental),16,0.053512,75.320913,1.0,0.05,299,LinGreedy (incremental) - 1.0
5,LinUCB (incremental),17,0.056856,17.926848,5.0,0.05,299,LinUCB (incremental) - 5.0
6,LinGreedy (incremental),16,0.053512,17.997318,5.0,0.05,299,LinGreedy (incremental) - 5.0
7,LinUCB (incremental),19,0.063545,9.397860,10.0,0.05,299,LinUCB (incremental) - 10.0
...,...,...,...,...,...,...,...,...
8,LinGreedy (incremental),589,0.071463,157.516431,10.0,0.95,8242,LinGreedy (incremental) - 10.0
9,LinUCB (incremental),506,0.061393,24.290873,100.0,0.95,8242,LinUCB (incremental) - 100.0
10,LinGreedy (incremental),605,0.073405,23.464792,100.0,0.95,8242,LinGreedy (incremental) - 100.0
11,LinUCB (incremental),485,0.058845,4.258888,1000.0,0.95,8242,LinUCB (incremental) - 1000.0


In [13]:
fig = px.line(df_results_incremental, x="test_size", y="hr", color='algo_batch', title='HR x Test size (incremental batch size)')
fig.show()
fig.write_html('results/hr_x_test_size_incremental_batch_size.html')

In [14]:
fig = px.line(df_results_incremental, x="test_size", y="time", color='algo_batch', title='Time x Test size (incremental batch size)')
fig.show()
fig.write_html('results/time_x_test_size_incremental_batch_size.html')

In [17]:
df_results_filtered = df_results[
    (df_results['algorithm'] == 'LinUCB (non_incremental)') | 
    (df_results['algorithm'] == 'LinGreedy (non_incremental)') |
    (df_results['algorithm'] == 'ALS') |
    ((df_results['algorithm'] == 'LinGreedy (incremental)') & (df_results['batch_size'] == 10.0))
]
df_results_filtered

,algorithm,hits,hr,time,batch_size,test_size,test_interactions
0,ALS,46,0.153846,0.250584,NaN,0.05,299
1,LinUCB (non_incremental),22,0.073579,0.078525,NaN,0.05,299
2,LinGreedy (non_incremental),22,0.073579,0.058038,NaN,0.05,299
8,LinGreedy (incremental),19,0.063545,10.021118,10.0,0.05,299
0,ALS,82,0.131621,0.294155,NaN,0.10,623
...,...,...,...,...,...,...,...
8,LinGreedy (incremental),630,0.074039,164.195556,10.0,0.90,8509
0,ALS,749,0.090876,2.593640,NaN,0.95,8242
1,LinUCB (non_incremental),556,0.067459,1.481422,NaN,0.95,8242
2,LinGreedy (non_incremental),556,0.067459,1.263177,NaN,0.95,8242


In [19]:
fig = px.line(df_results_filtered, x="test_size", y="hr", color='algorithm', title='HR x Test size')
fig.show()
fig.write_html('results/hr_x_test_size.html')